In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import sklearn.metrics
import matplotlib.pyplot as plt
import numpy as np
from kaggle.competitions import twosigmanews

In [ ]:
env = twosigmanews.make_env()

TODO: news_train_df も利用する

In [ ]:
(market_train_df, news_train_df) = env.get_training_data()

In [ ]:
market_train_df.describe()

In [ ]:
train_df = market_train_df.loc[:,["returnsClosePrevRaw1", "returnsOpenPrevRaw1", "returnsClosePrevMktres1", "returnsOpenPrevMktres1", "returnsClosePrevRaw10", "returnsOpenPrevRaw10", "returnsClosePrevMktres10", "returnsOpenPrevMktres10"]]

In [ ]:
target_df = market_train_df.loc[:,["returnsOpenNextMktres10"]]

In [ ]:
values = target_df.values

TODO: この分散を正規化していきたい
* Logだとマイナスの値が使えない
* 大きい数の外れ値があるけど、この外れ値は収入が大きかったときだからしっかり使いたい

In [ ]:
plt.hist(values) 

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(train_df, target_df)
dtrain = lgb.Dataset(train_x, label=train_y)
deval = lgb.Dataset(test_x, test_y, reference=dtrain)

In [ ]:
params = {'boosting': 'dart', 'num_leaves': 872, 'learning_rate': 0.0008514323478861367, 'drop_rate': 2.478486508669908e-05, 'skip_drop': 0.0007553027183709556 }    

In [ ]:
num_round = 191

~~TODO: ハイパーパラメータ設定~~ optunaを利用

In [ ]:
model = lgb.train(params, dtrain, valid_sets=deval)

TODO: confidenceValueのロジックを考える

In [ ]:
days = env.get_prediction_days()

In [ ]:
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    market_df = market_obs_df.loc[:,["returnsClosePrevRaw1", "returnsOpenPrevRaw1", "returnsClosePrevMktres1", "returnsOpenPrevMktres1", "returnsClosePrevRaw10", "returnsOpenPrevRaw10", "returnsClosePrevMktres10", "returnsOpenPrevMktres10"]]
    pred_y = model.predict(market_df, num_iteration=num_round)
    pred_y[pred_y > 1.0] = 1.0
    pred_y[pred_y < -1.0] = -1.0
    predictions_template_df.confidenceValue = pred_y
    env.predict(predictions_template_df)
print('Done!')

In [ ]:
env.write_submission_file()

In [ ]:
import os
print([filename for filename in os.listdir('.') if '.csv' in filename])